In [29]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma

In [30]:
def solve_1to1(Φ_i_j):
    n, m = np.shape(Φ_i_j)
    M_z_a = spr.bmat([[spr.kron(spr.eye(n), np.ones((1, m)))],
                      [spr.kron(np.ones((1, n)), spr.eye(m))]])
  
    q = np.concatenate((np.ones(n), np.ones(m)))

    model = grb.Model()
    mu_a = model.addMVar(n * m, vtype=grb.GRB.INTEGER)
    model.setObjective(Φ_i_j.flatten() @ mu_a, GRB.MAXIMIZE)
    model.addConstr(M_z_a @ mu_a <= q)
    
    model.Params.OutputFlag = 0
    model.optimize()

    return np.array(model.x, dtype= bool).reshape([n,m])*1

In [31]:
class OneToOneITU():
    def __init__(self, n, m, size_params, random_seed, lbs=(1, 1), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 

        # generate problem
        np.random.seed(random_seed)
        #self.A_ij = np.random.randint(1, size_params[0], size=[n, m])/  np.random.randint(1, size_params[0], size=[n, m])
        values_A = np.array([1,2,4,5,8,10,16,20])

        weight = np.random.randint(2, size=(self.n, self.m))
        self.A_ij = ( weight * np.random.choice(values_A[:size_params[0]],size= [n,m])  
                    + ( 1- weight) *(1/ np.random.choice(values_A[:size_params[0]],size= [n,m]) ))
        self.B_ij = np.random.randint(0, size_params[1], size=[n, m])

    def D_ij(self, U_i, V_j):
        return U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            U_ij = self.B_ij[i_idx,:] - self.A_ij[i_idx,:] * V_j[None, :], 
            out = self.lb_U * np.ones(np.shape(U_ij[0])[0])[:,None]
            return np.concatenate( ( U_ij[0], out  ) , axis= 1)
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

#### Checking modules

The following functions check equilibrium conditions for a candidate $(\mu, U, V).$. Below I will refer to the equilibrium conditions as follows:
* (*Primal Feasibility*) $\sum_{i} \mu_{ij} \leq 1$ and $\sum_{j} \mu_{ij} \leq 1 $ (and $\mu_{ij} \in \{0,1\}$),
* (*Primal Complementary Slackness*) $D_{ij}(U_i,V_j) = 0$ if $ \mu_{ij}  = 1$, 
* (*Dual Feasibility*) $D_{ij}(U_i,V_j) \geq 0$,
* (*Dual Complementary Slackness*) $U_i =  U_0 $ if $\sum_j \mu_{ij} = 0$ and $V_i =  V_0 $ if $\sum_i \mu_{ij} = 0,$ 
* (*Individual Rationality*) $U_i \geq  U_0 ,$ $V_i \geq  V_0 .$ 

**Dual Complementary Slackness**: $\forall i,j, \ U_i > U_{0} \implies \sum_j \mu_{ij} = 0, \ V_j > V_{0} \implies \sum_i \mu_{ij} = 0$

In [32]:
def check_dual_CS(self,U_i,V_j,mu_ij, output = False):

    D_CS_i = np.mean((U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1))
    D_CS_j = np.mean((V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1)) 
    display(Math(fr'\frac{1}{{n}} \sum_{{i}} (U_{{i}} - U_{{lb}} ) * (\sum_{{j}} \mu_{{ij}} -1) = {D_CS_i}'))
    display(Math(fr'\frac{1}{{m}}\sum_{{j}} (V_{{j}} - V_{{lb}} ) * (\sum_{{i}} \mu_{{ij}} -1) = {D_CS_j}'))
    
    if output is True:
        D_CS_i_bool = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        D_CS_j_bool = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return D_CS_i_bool, D_CS_j_bool
    
OneToOneITU.check_dual_CS = check_dual_CS

**Dual Feasibility**:  $\forall i,j,  \ A_{ij} U_i + G_{ij} V_j \geq B_{ij},$

In [33]:
def check_dual_feas(self,U_i,V_j, output = None):
 
    D_feas = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return D_feas, np.where(D_feas<0 )
    display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \min( A_{{ij}} U_i + G_{{ij}} V_j -  B_{{ij}} , 0)  = {D_feas.mean()}'))
    
   
OneToOneITU.check_dual_feas = check_dual_feas 

**Primal Complementary Slackness**:  $\forall i,j, \  \mu_{ij} * (A_{ij} U_i + G_{ij} V_j - B_{ij}) = 0 $

In [34]:
def check_primal_CS(self, U_i,V_j,mu_ij, output = None):

    P_CS_ij = self.D_ij(U_i,V_j) * mu_ij
    
    if output:
        return P_CS_ij
    else:
        display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \mu_{{ij}} * (A_{{ij}} U_i + G_{{ij}} V_j - B_{{ij}}) = {P_CS_ij.mean()}'))

OneToOneITU.check_primal_CS = check_primal_CS

**Primal Feasibility**:  $\forall i,j, \  \sum_j  \mu_{ij} \leq 1, \ \sum_i  \mu_{ij} \leq 1$

In [35]:
def check_primal_feas(self, mu_ij):

    display(Math(fr'\forall i, \ \sum_{{j}}  \mu_{{ij}} \leq 1: \ { np.all(np.sum(mu_ij,axis=1) <= np.ones(self.n))}'))
    display(Math(fr'\forall j, \ \sum_{{i}}  \mu_{{ij}} \leq 1: \ {np.all(np.sum(mu_ij,axis=0) <= np.ones(self.m)) }'))
    display(Markdown(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}"))

OneToOneITU.check_primal_feas = check_primal_feas

In [36]:
def check_IR(self, U_i,V_j):
    display(Math(fr'\min_{{i}} U_{{i}} = {np.min(U_i).round(3)} \geq {self.lb_U} = U_{{lb}}'))
    display(Math(fr'\min_{{j}} V_{{j}} = {np.min(V_j).round(3)} \geq  {self.lb_V} = V_{{lb}}'))

OneToOneITU.check_IR = check_IR

The following checks all conditions and prints the results

In [37]:
def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    header_size = 1

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Feasibility</h{header_size}>"))
    self.check_primal_feas(mu_ij)

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Generalized Complementary Slackness</h{header_size}>"))
    self.check_dual_feas(U_i, V_j )
    self.check_primal_CS(U_i, V_j , mu_ij)
    
    display(Markdown(f"{'_' * 4}\n<h{header_size}>Individual Rationality</h{header_size}>"))
    self.check_IR(U_i, V_j )
    self.check_dual_CS(U_i, V_j , mu_ij)

OneToOneITU.check_all = check_all

# Algorithm

### inner routines

In [38]:
def iteration(self,i_ent, mu_, V_j_):
    mu = mu_.copy()
    V_j = V_j_.copy()

    unassigned_js = np.concatenate((mu[:,:-1].sum(0) == 0,[True]))

    C  = np.zeros(self.n, dtype= bool)
    C_i_ent = np.zeros(self.n, dtype= bool)
    C[i_ent] = 1
    B = np.ones(self.m, dtype= bool)
    L_j = np.ones(self.m, dtype= int) *(self.n + 1)
    d_j = np.ones(self.m + 1, dtype= int) * np.inf

    iter = 0
    while np.any(C == 1):
        iter += 1
        i = np.argmax(C)
        #print(f"iter {iter}, i: {i}")
        C[i] = 0
        C_i_ent[i] = 1
        U_ij_s = self.get_U_ij(V_j, i)[0]
        π_i = np.max(U_ij_s)
        #print(π_i)

        suboptimal_js = U_ij_s < π_i 
      

        if np.all(suboptimal_js[:-1] == 1):
             mu[i,-1] = 1
             return mu, V_j
        
        A_ij_star_max = np.max(self.A_ij[i,~(suboptimal_js[:-1] )])
        d_j[suboptimal_js]= np.minimum(d_j[suboptimal_js], (π_i - U_ij_s[suboptimal_js]) /A_ij_star_max) 

       
        unissigned_optimal_js = np.where( unassigned_js  * (π_i == U_ij_s))[0]
    
        if len(unissigned_optimal_js)> 0: #####
            i_t = i
            j_t = unissigned_optimal_js[0]
          
            while i_t != i_ent :
 
                j_t_plus_1 = np.argmax(mu[i_t,: ])
                mu[i_t,: ] = 0
                mu[i_t, j_t] = 1
                j_t = j_t_plus_1
                i_t = L_j[j_t]
         
            w_i_s = np.sort( self.get_U_ij(V_j, i_t)[0])[-2]
            mu[i_t,: ] = 0
            mu[i_t, j_t] = 1
            if j_t < self.m:
                V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)
            #print("augmentation")
            return mu, V_j
            
        for j in np.where(B * (~suboptimal_js[:-1]) >0)[0]:
                i_j = np.argmax(mu[:,j])
                B[j] = 0
                C[i_j] = 1
                L_j[j] = i if L_j[j] == self.n + 1 else None
                #print(f"j : {j}")
                #print(L_j)
                #print(f"i:{i} -> j:{j} ")
    
     
    return C_i_ent, B * (d_j[:-1] < np.inf), d_j, iter

OneToOneITU.iteration = iteration

In [39]:
def interim_auction(self,C, O, out_opt_i, V_j_, mu_ ):
    V_j = V_j_.copy()
    pi_ij = np.concatenate((mu_[C,:-1][:,O], np.zeros(C.sum(), dtype= bool)[:,None] ), axis= 1)

        

    U_ij = np.concatenate((self.get_U_ij(V_j, C)[:, :-1][:,O], out_opt_i[:,None]), axis= 1)
    sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
    w_i = U_ij[np.arange(C.sum()),sorted_w_id[:,0] ]
    π_i = np.max(U_ij, axis = 1)
    ij_optimal = U_ij == π_i[:, None]
    offers = ((π_i - w_i)[:,None]/ self.A_ij[C,:][:,O] ) * ij_optimal[:,:-1]


    ij_optimal[:,-1] = 0
    pi_ij = solve_1to1(ij_optimal)

    offers_coop = ((U_ij[:,:-1] - out_opt_i[:,None])/ self.A_ij[C,:][:,O] ) 


    offers_coop = np.min(offers_coop[offers_coop > 0], axis= 0)
    j_low = offers_coop == np.min(offers_coop)
    V_j[O] += j_low * np.min(offers_coop)
        

    U_ij = np.concatenate((self.get_U_ij(V_j, C)[:, :-1][:,O], out_opt_i[:,None]), axis= 1)
    π_i = np.max(U_ij, axis = 1)
    ij_optimal = U_ij == π_i[:, None]
    ij_optimal[:,-1] = 0
    pi_ij = solve_1to1(ij_optimal)
    return pi_ij, V_j

OneToOneITU.interim_auction =interim_auction

### Notes

In [40]:
# n_small = 5
# m_small = 5
# example_small = OneToOneITU( n = n_small,m = m_small, size_params= (3,4) , random_seed= 1586, lbs =(0,0), tol = 0)

In [41]:
mu_ex = np.zeros([n_small,m_small+1])
V_ex = np.ones(example_small.m) * example_small.lb_V

NameError: name 'n_small' is not defined

In [ ]:
#C_i = np.zeros(example_small.n, dtype= bool)
for p in range(1):
    i_ent = np.where(mu_ex.sum(1) == 0)[0][0]
    iter_values = example_small.iteration(i_ent, mu_ex, V_ex )
    #print(iter_values[1])
    V_ex_pre = V_ex.copy()
    mu_ex_pre = mu_ex.copy()
    if len(iter_values) == 2:
        mu_ex, V_ex  = iter_values
        print("augment")
 
    else:

        C_i, B, d ,j_low= iter_values
        U_ij = example_small.get_U_ij(V_ex, C_i)
        val_i = np.max(U_ij, axis = 1)

        w_i_out_B = np.max(U_ij[:,np.concatenate((B,[True]))], axis = 1)

       
        #V_ex[~B] += np.min(d[B_with_out])
        pi_ij, V_ex = example_small.interim_auction(C_i, ~B,w_i_out_B ,V_ex, mu_ex )
        id_O = 0
        for j in np.where( B == 0 )[0]:
            mu_ex[C_i,j] = pi_ij[:,id_O]
            id_O += 1
        

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [ ]:
mu_ex

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [ ]:
example_small.get_U_ij(V_ex, np.ones(example_small.n, dtype=bool))

array([[  0. ,   4. ,  -5. ,   5. ,   3. ,   1. ],
       [  1. ,   0. ,   2. , -11. ,   0. ,   1. ],
       [  2. ,   3.5,   3.5,  -4. ,   0. ,   1. ],
       [  5. ,   4. ,  -4. ,  -1. ,   3. ,   1. ],
       [  0. ,   5. ,   1.5,   5. ,   2. ,   1. ]])

In [ ]:
V_ex

array([4. , 0.5, 7. , 8. , 0. ])

In [ ]:
for i in np.where(mu_ex[:,-1] >0)[0]:
    mu_ex[i,:] = 0
    mu_ex, V_ex = example_small.iteration(0, mu_ex, V_ex )

In [ ]:
example_small.iteration(0, mu_ex, V_ex )

(array([[0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]),
 array([0., 0., 0., 0., 0.]))

# Coop auction

In [64]:
n_small = 25
m_small = 25
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (3,20) , random_seed= 634, lbs =(1,0), tol = 0)

In [65]:
example_small.A_ij

array([[0.25, 1.  , 0.25, 0.25, 1.  , 1.  , 4.  , 1.  , 4.  , 1.  , 0.25,
        0.5 , 0.5 , 1.  , 4.  , 0.5 , 4.  , 2.  , 0.25, 1.  , 1.  , 4.  ,
        0.5 , 0.25, 0.5 ],
       [2.  , 2.  , 1.  , 1.  , 0.5 , 1.  , 1.  , 0.25, 2.  , 2.  , 1.  ,
        0.5 , 0.25, 1.  , 1.  , 0.5 , 2.  , 1.  , 2.  , 2.  , 0.5 , 1.  ,
        4.  , 0.25, 0.5 ],
       [1.  , 0.5 , 1.  , 4.  , 0.5 , 4.  , 0.5 , 4.  , 1.  , 1.  , 0.25,
        0.25, 2.  , 4.  , 1.  , 0.25, 1.  , 0.25, 1.  , 4.  , 1.  , 0.5 ,
        4.  , 2.  , 0.5 ],
       [1.  , 4.  , 1.  , 0.25, 4.  , 0.25, 2.  , 2.  , 4.  , 1.  , 4.  ,
        4.  , 4.  , 2.  , 0.5 , 0.5 , 2.  , 0.5 , 4.  , 0.25, 0.25, 1.  ,
        1.  , 0.5 , 4.  ],
       [1.  , 1.  , 2.  , 1.  , 1.  , 0.25, 1.  , 2.  , 4.  , 1.  , 0.25,
        2.  , 0.25, 0.25, 4.  , 1.  , 0.5 , 1.  , 0.25, 1.  , 1.  , 1.  ,
        0.25, 1.  , 0.5 ],
       [0.5 , 0.5 , 1.  , 1.  , 4.  , 0.5 , 1.  , 2.  , 1.  , 1.  , 2.  ,
        1.  , 1.  , 0.25, 1.  , 4.  , 1.  , 1.  , 0

In [66]:
def coop_ITU_auction(self):
    mu_ij = np.zeros([self.n,self.m+1], dtype= bool)
    V_j = np.ones(self.m) * self.lb_V
    while np.any( mu_ij.sum() < np.minimum(self.n, self.m)):
    # for p in range(5):
        
        i_ent = np.where(mu_ij.sum(1) == 0)[0][0]
        iter_values = self.iteration(i_ent, mu_ij, V_j )
        #print(iter_values[1])
        V_j_pre = V_j.copy()
        mu_ij_pre = mu_ij.copy()
        if len(iter_values) == 2:
            mu_ij, V_j  = iter_values
            #print("augment")    
        else:

            C_i, B, d ,j_low= iter_values
            U_ij = self.get_U_ij(V_j, C_i)
            val_i = np.max(U_ij, axis = 1)
            w_i_out_B = np.max(U_ij[:,np.concatenate((B,[True]))], axis = 1)
    
            pi_ij, V_j = self.interim_auction(C_i, ~B,w_i_out_B ,V_j, mu_ij )
            id_O = 0
            for j in np.where( B == 0 )[0]:
                mu_ij[C_i,j] = pi_ij[:,id_O]
                id_O += 1


    U_ij = self.get_U_ij(V_j, np.ones(self.n, dtype= bool))
    U_i = np.max(U_ij, axis= 1 )
    # print(U_ij)
    #print( (U_ij[:,:-1] == U_i[:,None] )*1)
    opt_ij = np.array((U_ij == U_i[:,None] ), dtype= int)
    js_slack = list(V_j > self.lb_V)
    js_slack.append(False)
    opt_ij[:,js_slack] *= 2
    # is_slack = U_i > self.lb_U
    # opt_ij[,:] = 2
    #a[:,:-1][V_j > self.lb_V] = 2

    # print(opt_ij)
    mu_ij = solve_1to1(opt_ij)

    return mu_ij[:,:-1], U_i, V_j

OneToOneITU.coop_ITU_auction = coop_ITU_auction

In [67]:
mu_ex , U_ex,  V_ex  = example_small.coop_ITU_auction()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coeffici

In [59]:
# U_ex

In [60]:
# example_small.get_U_ij(V_ex, np.ones(example_small.n, dtype=bool))

In [61]:
# V_ex

In [62]:
# id_i , id_j  =  np.where(mu_ex[:,:-1] == 1)
# U_ex = np.ones(example_small.n) * example_small.lb_U
# U_ex[id_i] = example_small.get_U_ij(V_ex, id_i)[np.arange(len(id_i)), id_j]  

In [68]:
example_small.check_all((mu_ex , U_ex , V_ex ))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 25 over 25

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Large simulation

In [37]:
violations = []

for iter in range(10000):
    print(iter)
    n_small = 10
    m_small = 10
    example_simul = OneToOneITU( n = n_small,m = m_small, size_params= (3,10) , random_seed= iter, lbs =(0,0), tol = 0)
    mu_sim , U_sim,  V_sim  = example_simul.coop_ITU_auction()
    # print(example_simul.B_ij[0,:])
    
    feas_dual = example_simul. check_dual_feas(U_sim ,V_sim, output = True)[0].sum() 
    CS = example_simul.check_primal_CS(U_sim, V_sim, mu_sim, output= True).sum()
    IR = ((mu_sim.sum(0) == 0) * ( V_sim - example_simul.lb_V)).sum()   
    IR_i = ((mu_sim.sum(1) == 0) * ( U_sim - example_simul.lb_U)).sum() 
    feas = np.all(mu_sim.sum(0)<=1) * np.all(mu_sim.sum(1)<=1)
   
    if CS != 0 or IR != 0 or ~feas or feas_dual!= 0 or IR_i != 0:
        print("porcaccio")
        print(IR)
        print(CS)
        print(feas_dual)
        print(f"iter{iter}")
        violations.append((iter,CS,feas_dual,IR, IR_i ))

0
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
1
2
3
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
4
5
6
7
8
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


9
10
11
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
12
13
14
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
15
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
16
17
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero o

In [1]:
len(violations)

NameError: name 'violations' is not defined

In [39]:
violations

[]